In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

2025-05-05 20:55:56.054121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746478556.254279      18 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746478556.309037      18 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    shear_range=0.2,
    zoom_range=0.2,
    rescale=1/255.0,
    fill_mode='nearest',
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,  
    brightness_range=[0.8, 1.2],
    vertical_flip=False,
)

val_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0,
)

train_gen = data_gen.flow_from_directory(
    directory="/kaggle/input/brain-tumor-mri-dataset/Training",
    target_size = (224,224),
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    color_mode='rgb',
)

test_gen = val_gen.flow_from_directory(
    directory = "/kaggle/input/brain-tumor-mri-dataset/Testing",
    target_size = (224,224),
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    color_mode='rgb',
)

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Reshape, Dropout, BatchNormalization, GlobalAveragePooling1D, LayerNormalization, Add
import numpy as np

# Cyclical Learning Rate Scheduler (Fixed)
class CyclicalLRScheduler(tf.keras.callbacks.Callback):
    def __init__(self, base_lr, max_lr, step_size):
        super(CyclicalLRScheduler, self).__init__()
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size

    def on_batch_begin(self, batch, logs=None):
        cycle = np.floor(1 + batch / (2 * self.step_size))
        x = np.abs(batch / self.step_size - 2 * cycle + 1)
        lr = self.base_lr + (self.max_lr - self.base_lr) * max(0, (1 - x))
        # Directly assign the new learning rate to the optimizer
        self.model.optimizer.learning_rate.assign(lr)

# Build Model
input_layer = Input(shape=(224, 224, 3))
base_model = DenseNet121(include_top=False, weights='imagenet', input_tensor=input_layer)
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x_1 = Reshape((1, -1))(x)
embed_dim = 512
x_1 = Dense(embed_dim)(x_1)
x_residual = x_1
x_1 = LayerNormalization(epsilon=1e-6)(x_1)

mha = layers.MultiHeadAttention(num_heads=8, key_dim=embed_dim // 8)
x_att = mha(query=x_1, value=x_1, key=x_1)
x_att = LayerNormalization(epsilon=1e-6)(x_att)
x_att = Add()([x_att, x_residual])

x_ffn = Dense(embed_dim, activation='relu')(x_att)
x_ffn = Dense(embed_dim)(x_ffn)
x_ffn = LayerNormalization(epsilon=1e-6)(x_ffn)
x = Add()([x_att, x_ffn])
x = GlobalAveragePooling1D()(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(32, activation="relu")(x)
output = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, clipnorm=1.0),
    metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
)

early_stopping = tf.keras.callbacks.EarlyStopping(patience=8, monitor='val_loss', mode='min', restore_best_weights=True)
cyclical_lr = CyclicalLRScheduler(base_lr=1e-6, max_lr=1e-4, step_size=200)
model.fit(
    train_gen, epochs=10, batch_size=32, validation_data=test_gen,
    callbacks=[early_stopping, cyclical_lr]
)

I0000 00:00:1746478573.188585      18 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 8, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1746478613.692495      73 service.cc:148] XLA service 0x7df4e0002240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746478613.693427      73 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00

  2/179 ━━━━━━━━━━━━━━━━━━━━ 11s 63ms/step - accuracy: 0.1797 - loss: 1.7593 - precision: 0.2386 - recall: 0.0859   

I0000 00:00:1746478634.819466      73 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


179/179 ━━━━━━━━━━━━━━━━━━━━ 200s 818ms/step - accuracy: 0.3797 - loss: 1.4183 - precision: 0.4460 - recall: 0.2269 - val_accuracy: 0.7765 - val_loss: 0.5772 - val_precision: 0.8188 - val_recall: 0.7445
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 79s 423ms/step - accuracy: 0.7302 - loss: 0.7056 - precision: 0.8204 - recall: 0.6116 - val_accuracy: 0.8261 - val_loss: 0.4809 - val_precision: 0.8498 - val_recall: 0.7941
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 79s 426ms/step - accuracy: 0.8193 - loss: 0.5358 - precision: 0.8732 - recall: 0.7399 - val_accuracy: 0.8268 - val_loss: 0.4553 - val_precision: 0.8547 - val_recall: 0.8032
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 79s 424ms/step - accuracy: 0.8547 - loss: 0.4596 - precision: 0.8860 - recall: 0.7983 - val_accuracy: 0.8162 - val_loss: 0.4597 - val_precision: 0.8423 - val_recall: 0.8024
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 79s 423ms/step - accuracy: 0.8628 - loss: 0.4310 - precision: 0.8929 - recall: 0.8279 - val_accuracy: 0.8802 - val_los